<a></a>
<div style="border-radius: 10px; border: 1px solid #0F9CF5; background-color: #232323; white-space: nowrap;">
    <p style="margin-top: -10px; margin-bottom: 0px; margin-left: 10px; font-size: 1.15em; padding: 10px; overflow: hidden;">
        <span style="color: orange; font-size: 2em;">&#9432;  </span>
        Click the <span style="color: orange;">Run All</span> <img style="max-height: 1.5em; border: 1px solid orange;" src="../img/RunAll.png" /> button in the toolbar above to run the code in this notebook 
    </p>
</div>

<a id="document-top"></a>
# BQuant Machine Learning Series Part 7 - K - Means Clustering


<a href='https://onlinexperiences.com/scripts/Server.nxp?LASCmd=AI:4;F:QS!10100&ShowUUID=66209920-766E-492D-9C3D-914AE5426CC6'>Link to Episode 7 - ML Series Video - K-Means Clustering</a>

In [ ]:
import numpy as np
import pandas as pd
import bqviz
import ipywidgets as widgets 
from scipy.stats import mstats
from sklearn.cluster import KMeans
import bql

<p>
    In this notebook, we will explore K-Means Clustering:<br>
    <ul>
        <li><a href="#definition"> What is K-Means Clustering? </li>
        <li><a href="#algorithm">Algorithm</a></li>
        <li><a href="#application">K-Means Clustering Application: Building a diversified portfolio</a></li>
    </ul>
</p>

<a name="definition"></a>
<h2><span style="color:orange">What is K-Means Clustering?  </span></h2>
<p>
K-Means Clustering is a form of unsupervised ML. It is considered as one of the simplest and popular unsupervised machine learning techniques.
Unsupervised algorithms use vectors on data points. These data points are not labeled or classified. Our goal is to discover hidden patterns and group the data points in a sensible way based on similarity on features. Each group of datapoints is a cluster and each cluster will have a center.

<img src="img/k_means.png">

<a href="https://www.analyticsvidhya.com/blog/2021/04/k-means-clustering-simplified-in-python/">Source : Analytics Vidhya </a>

### Examples

#### Row data

<img src="img/k_means_row_data.png">

#### Good Clustering

<img src="img/k_means_good_clustering.png" >

#### Naive Clustering

<img src="img/k_means_bad_clustering.png" >

Source: Oreilly.com : Clustering and Unsupervised Learning

<a name="algorithm"></a>
<h2><span style="color:orange">Algorithm </span></h2>

* Pre-process the data (Clean it, Scale it, Standardize it)
* Select K
* Pick K Centers 
* Repeat until there is no change of the centroid positions: <BR>
   1) Compute the distance between data point (vector x) and all centroids. (Generally, we use the euclidian distance) $$Distance(k)=\sqrt{\sum \limits _{i}(x_{i} - c_{i}) ^2} \; for \, k=1 \, to \, K $$ 
   2) Assign each data point to the closest cluster (centroid) $$\underset{k}{\operatorname{argmin}} Distance(k)$$
   3) Compute the centroids for the clusters by taking the average of the all data points that belong to each cluster.
 

 <img src="img/k_means_algorithm.png"  width="40%" >

<a href="https://stanford.edu/~cpiech/cs221/handouts/kmeans.html">Source : Stanford Edu ( K-Means) </a>


<a name="application"></a>
<h2><span style="color:orange">K-Means Clustering Application: Building a diversified portfolio </span></h2>

We are going to use K-Means Clustering to build a diversified portfolio. Two ratios will be used in order to cluster the data: <BR>
<ul>
    <li> <code>asset_turnover:</code> Amount of sales or revenues generated per dollar of assets. (Sales Revenues/Total Assets)</li>
    <li><code>return_on_asset:</code> Indicator of how profitable company is relative to its assets (Total Income /Total Assets)</li>

</ul>
The idea is to create clusters with similar characteristics for the components of S&P500 using these two factors at the end of the 2021 Q1. From each cluster, we will take the stocks with highest risk adjusted momentum to build our portfolio. <br><br>
   After building this portfolio, we will run it for 2021 Q2 and compare it to the return of S&P 500.<br><br>
    
PS: Please note that this analysis is done using only two factors which leads to a two dimensional problem. We are using a two dimensional problem to demonstrate the concept and understand the problem. Multiple factors can be used as well. If you want to go down this road, you may want to use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html">Principal Component Analysis</a> to lower the number of dimensions. <br><br>
   
We will proceed with the following steps: <br><b>K-Means Clustering : </b><br>

    1. Get the data: asset_turnover and return_on_asset for the end of 2021 Q1 for members of S&P500.
    2. Analyse the data, clean it and vizualize it.
    3. Choose K.
    4. Analyse the clustering results.
<b>Portfolio Construction : </b><br>
    
    1. From each cluster, choose the stocks with the highest risk adjusted momentum. 
    2. Run the portfolio return for the 2021-Q2

----
Let's apply the steps defined above:
## K-Means Clustering 
### <I>1. Get the data: asset_turnover and return_on_asset for the end of 2021 Q1 for members of S&P500</I>

In [ ]:
bq = bql.Service()

as_of_date='2021-03-30'

asset_turnover= bq.func.dropna(bq.data.asset_turnover(fa_period_type='Q',fill='prev',as_of_date=as_of_date)['Value'])
return_on_asset =bq.func.dropna(bq.data.RETURN_ON_ASSET(fa_period_type='Q',fill='prev',as_of_date=as_of_date)['Value']/100)

#  We are requesting the members of SPY US Equity since some users don't have access to the members of S&P 500. 
#  If you have access to the members of S&P500 you can request that direcly using bql_universe=bq.univ.members('SPX Index')
bql_universe = bq.univ.members('SPY US Equity',type='HOLDINGS')

bql_request = bql.Request(bql_universe, {
                                         'asset_turnover':asset_turnover,
                                         'return_on_asset':return_on_asset}
                         )
bql_response = bq.execute(bql_request)
data=bql.combined_df(bql_response)
data.head()

### <I>2. Analyse the data, clean it and visualize it.</I>

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
# Make a copy of the original data before starting our data pre processing
original_data=data.copy()

In [ ]:
#Check Na Values
data[data['asset_turnover'].isna() | data['return_on_asset'].isna()]

In [ ]:
# Dropna value 
data=data.dropna()
#remove Partial Error Column
data.drop('Partial Errors',axis=1,inplace=True)
data.head()

In [ ]:
# Viuzualize data
plot=bqviz.InteractiveScatterPlot(data,title='Original Data', reg_line=False)
plot.x_control.value =  'asset_turnover' 
plot.y_control.value = 'return_on_asset'
plot.show()

In [ ]:
# Both asset_turnover and return_on_asset are ratios. They are already scaled to the company size.
# We can use Winsorization to transforms data by limiting extreme values, typically by setting all outliers to a specified percentile of data
X =np.asarray([np.asarray(data['asset_turnover']),np.asarray(data['return_on_asset'])])
X = mstats.winsorize(X, limits = [0.05, 0.05])
data=pd.DataFrame(X, index=['asset_turnover','return_on_asset'], columns=data.index).T
data.head()

In [ ]:
# Plot winsorized data Vs Original Data
plot_winsorized=bqviz.InteractiveScatterPlot(data,title='Winsorized Data', reg_line=False)
plot_winsorized.x_control.value =  'asset_turnover' 
plot_winsorized.y_control.value = 'return_on_asset'
widgets.HBox([plot_winsorized.show(),plot.show()])

### <I>3. Choose K</I>

The two most commons methods to choose K ( the appropriate number of clusters) are :
    <ul>
        <li>The silhouette Coefficient</li>
        <li>The Elbow Method </li>
    </ul>

The silhouette coefficient is a value that ranges between -1 and 1. It quantifies how well a data point fits into its assigned cluster based on two factors:
1. How close the data point is to other points in the cluster
2. How far away the data point is from points in other clusters

Larger numbers for Silhouette coefficient indicate that samples are closer to their clusters than they are to other clusters.

The elbow method is used by running several k-means, increment k with each iteration and record the SSE ( Sum Of Squared Error) <br>
$$SSE= Sum  \; Of  \; Euclidean  \; Squared  \; Distances  \; of  \; each  \; point \; to \; its  \; closest \; centroid $$
After that , we plot SSE as a function of the number of clusters. SSE continues to decrease as you increase k. As more centroids are added, the distance from each point to its closest centroid will decrease.
There’s a sweet spot where the SSE curve starts to bend known as the elbow point. The x-value of this point is thought to be a reasonable trade-off between error and number of clusters. <br>

<a herf="https://realpython.com/k-means-clustering-python/#choosing-the-appropriate-number-of-clusters"> (Source)<a/>
    
In this example, we will use the Elbow Method to determine K:

In [ ]:
distorsions = []
clusters_iterations=range(2, 20)
for k in clusters_iterations:
    k_means = KMeans(n_clusters=k)
    k_means.fit(data)
    distorsions.append(k_means.inertia_)

In [ ]:
elbow_curve_data=pd.DataFrame(zip(clusters_iterations,distorsions),columns=['Cluster','SSE']).set_index('Cluster')
elbow_curve_data.head()

In [ ]:
plot_elbow=bqviz.LinePlot(elbow_curve_data,title='Elbow Curve',x_label='Clusters').set_style()
plot_elbow.show()

In [ ]:
# get elbow programmatically
%install kneed 0.7.0
from kneed import KneeLocator 
kl = KneeLocator(
clusters_iterations, distorsions, curve="convex", direction="decreasing")
elbow=kl.elbow

print('Elbow = {}'.format(elbow))

### <I>4. Analyse the clustering results</I>

In [ ]:
# We apply KMeans for the Elbow's value  ( in this case = 5)
kmeans = KMeans(n_clusters=elbow)
kmeans.fit(data)
y_kmeans = kmeans.predict(data)
df_kmeans = data.copy()
df_kmeans['cluster']=y_kmeans.astype(str)

In [ ]:
# Vizaualize the results
plot_kmeans=bqviz.InteractiveScatterPlot(df = df_kmeans,color_field='cluster',scheme='Red', title='K-Means Clustering')
plot_kmeans.x_control.value =  'asset_turnover' 
plot_kmeans.y_control.value = 'return_on_asset'
plot_kmeans.show()

In [ ]:
# see the centers 
clusters_centers_df=pd.DataFrame(kmeans.cluster_centers_,columns=['asset_turnover','return_on_asset'])
clusters_centers_df

In [ ]:
# See the clustering by Company 
clustering_result=pd.DataFrame(zip(y_kmeans,data.index),columns=['Cluster','Company'])
clustering_result.set_index('Cluster').head()

In [ ]:
for cluster_num in list(clustering_result.set_index('Cluster').index.unique()):
    print (clustering_result.set_index('Cluster').loc[cluster_num].head())

In [ ]:
# Enrich Centers Df with the number of elements by Cluster
clusters_centers_df['Count']=clustering_result['Cluster'].value_counts().to_frame().rename(columns={'Cluster':'Count'})['Count']
clusters_centers_df.head()

In [ ]:
# Vizualize Count of Elements by Cluster 
plot_cluster_count=bqviz.BarPlot(clusters_centers_df['Count'].to_frame()).set_style()
plot_cluster_count.show()

## Portfolio Construction
### <I>1. From each cluster, choose the stocks with the highest Risk Adjusted Momentum </I>

In [ ]:
# Build of Portfolio of 50 stocks
number_of_stocks=50

# From each Cluster, we will pick the stocks with the highest risk adjusted momentum. The number of stocks from each cluster is proportional to its size
# Let's start by calculate the number of stocks to pick from each cluster
number_of_stocks_by_cluster=pd.DataFrame(round(number_of_stocks*clustering_result.groupby(by='Cluster').count()['Company']/clustering_result.count()['Company'],0))
number_of_stocks_by_cluster

In [ ]:
# Now we construct our portfolio, 

# initialise the portfolio 
portfolio_stocks=[]
for cluster_num in list(number_of_stocks_by_cluster.index):
    # for each cluster,get all the companies within this cluster
    list_stocks=list(clustering_result.set_index('Cluster').loc[cluster_num]['Company'])
    #get the number of stocks that we will pick for our portfolio     
    number_stocks=number_of_stocks_by_cluster.loc[cluster_num]['Company']
    if number_stocks>0:
        # Compute the risk adjusted momentum for the past year
        last_year_date=pd.to_datetime(as_of_date)+ pd.offsets.DateOffset(years=-1)
        last_month_date=pd.to_datetime(as_of_date)+ pd.offsets.DateOffset(months=-1)
        annual_volatility = bq.func.std(bq.func.diff(bq.func.ln(bq.func.dropna(bq.data.px_last(dates=bq.func.range(start=last_year_date, end=as_of_date)))))) * (bq.func.sqrt(252))['Value']
        #12month-1month momentum/annualized vol         
        risk_adjusted_momentum = (bq.func.pct_chg(bq.data.px_last(dates=bq.func.range(start=last_year_date, end=last_month_date), fill='prev'))/annual_volatility)['Value']
        
        # Request only the top stocks with the hishest risk adjusted momentum
        bql_item = bq.func.groupsort(risk_adjusted_momentum)
        bql_universe = bq.univ.filter(bq.univ.list(list_stocks), bq.func.grouprank(risk_adjusted_momentum) <= number_stocks)
        bql_request = bql.Request(bql_universe, bql_item)
        bql_response = bq.execute(bql_request)
        
        # concat the stocks retrieved from this cluster to our portfolio
        portfolio_stocks=portfolio_stocks+list(bql_response[0].df().index)
portfolio_stocks

### <I> 2. Compute Portfolio's Performance for 2021-Q2 </I>

In [ ]:
# Since we choosed our portfolio stocks by end the of 2021-Q1, we will run it for 2021-Q2 
end_date='2021-06-30'

# Compute the portfolio return. We will use equal weights for all the stocks
daily_return=bq.func.dropna(bq.data.day_to_day_tot_return_gross_dvds(dates=bq.func.range(start=as_of_date, end=end_date)))+1
cluster_portfolio_return=0
for stock in portfolio_stocks:
    cluster_portfolio_return=cluster_portfolio_return+bq.func.value(daily_return,bq.univ.list(stock))/len(portfolio_stocks)
bql_universe = 'SPX Index'
bql_request = bql.Request(bql_universe, {'cluster_portfolio_return':cluster_portfolio_return,
                                         'spx_index_return':daily_return,
                                        })
bql_response=bq.execute(bql_request)
return_ptf_index=bql.combined_df(bql_response)
return_ptf_index=return_ptf_index.set_index('DATE').drop('ORIG_IDS',axis=1)
return_ptf_index.head()

In [ ]:
from math import sqrt
# Compute the annual violatility, sharpe ratio and annual excess return and plot the cum return

# compute the timeline for annualization
T = (return_ptf_index['cluster_portfolio_return'].index[-1] - return_ptf_index['cluster_portfolio_return'].index[0]) / np.timedelta64(1, 'Y')

#portfolio Excess Return
portfolio_excess_return=round(100*(return_ptf_index['cluster_portfolio_return'].cumprod().iloc[-1]**(1/T) - 1),2)

#Portfolio Annual Volatility
portfolio_annual_volatility=round(100*return_ptf_index['cluster_portfolio_return'].std()*sqrt(252),2)

#Portfolio Sharpe Ratio
portfolio_sharpe_ratio=round((portfolio_excess_return)/portfolio_annual_volatility,2)

# Plot Results
print ("Portfolio Annual Excess Return : {}%".format(portfolio_excess_return))
print ("Portfolio Annual Volatility    : {}% ".format(portfolio_annual_volatility))
print ("Portfolio Sharpe Ratio         : {}".format(portfolio_sharpe_ratio)) 
plot_ptf_performance=bqviz.InteractiveLinePlot(df = return_ptf_index.cumprod(), legend= 'outside')
plot_ptf_performance.show()

You can repeat this analysis in order to build a portfolio that rebalances every end of Quarter. Please be mindful to the amount of data retrieved using your BQL Queries in order to avoid data limit issues.
<h3>References and Additional Resources</h3>


<ul>
        
  <li><a href ="https://www.cs.princeton.edu/sites/default/files/uploads/karina_marvin.pdf"> Princeton University: Creating Diversified Portfolios Using Cluster Analysis </a></li>
 <li> <a href ="https://scholarship.claremont.edu/cgi/viewcontent.cgi?article=3517&context=cmc_theses"> Scholarship @ Claremont :K-Means Stock Clustering Analysis Based on Historical Price Movements and Financial Ratios  </a> </li>
 <li> <a href ="https://realpython.com/k-means-clustering-python/"> Real Python: K-Means Clustering in Python: A Practical Guide  </a> </li>
</ul>